# Limpeza e preparação de dados para análise

Este notebook tem a por finalidade a preparação de um DataSet para posterior análise de dados.

In [597]:
# importando bibliotecas
import pandas as pd
import numpy as np
from summarytools import dfSummary

## Carregando os dados

In [598]:
# Lendo os dados
open_masc = pd.read_csv('data/men_main_21.1.csv', sep=',', low_memory=False)

## Visualizando dados
Essa visualização é importante para compreender o que precisa ser feito no dataset.

Verifica-se que algumas colunas de valores estão em string e precisam ser convertidos para numéricos para possibilitar que os calculos sejam feitos.

Há uma coluna que condensa dois dados, peso e altura do atleta, deverá ser convertida em duas colunas.

Há outras colunas que condensam duas informações, por exemplo, a coluna '21.1' armazena dados da posição do atleta naquela prova e também do tempo em que o atleta concluiu a prova. *Vamos ficar apenas com os dados do tempo, ignorando a posição do atleta na prova*.

In [599]:
# Primeira visualização do Dataframe
open_masc.head()

,Unnamed: 0,first_name,lastname,Placing,country,continent,Age,affiliate,Height and Weight,points,21.1,21.1 time,21.2,21.2 time,21.3,21.3 time,21.4,weight,Unnamed: 18
0,0,JEFFRE,ADLER,1,Canada,North America,Age 27,CrossFit Wonderland,69 in | 197 lb,101.0,20th (11:55),605 reps,8th (9:14),225 reps,27th (8:15),180 reps,46th (317 lbs),Weight lifted: 317 lbs.,NaN
1,1,SCOTT,PANCHIK,2,United States,North America,Age 33,CrossFit Mentality,69 in | 187 lb,141.0,33rd (12:25),605 reps,47th (9:52),225 reps,5th (7:48),180 reps,56th (316 lbs),Weight lifted: 316 lbs.,NaN
2,2,TRAVIS,MEAD,3,United States,North America,Age 34,Iron Valley CrossFit,73 in | 205 lb,165.0,87th (13:02),605 reps,24th (9:38),225 reps,48th (8:26),180 reps,6th (345 lbs),Weight lifted: 345 lbs.,NaN
3,3,SAXON,PANCHIK,4,United States,North America,Age 25,CrossFit Cliffside,69 in | 180 lb,217.0,5th (11:25),605 reps,68th (9:59),225 reps,87th (8:41),180 reps,57th (316 lbs),Weight lifted: 316 lbs.,NaN
4,4,RICHARD,FRONING JR.,5,United States,North America,Age 33,CrossFit Mayhem,69 in | 194 lb,254.0,58th (12:45),605 reps,91st (10:04),225 reps,5th (7:48),180 reps,100th (312 lbs),Weight lifted: 312 lbs.,NaN


In [600]:
# describe() with lasers!
dfSummary(open_masc)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,Unnamed: 0[int64],Mean (sd) : 68731.5 (39682.6)min < med < max:0.0 < 68731.5 < 137463.0IQR (CV) : 68731.5 (1.7),"137,464 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAHoAAAAyCAYAAACTUs/lAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAAsTAAALEwEAmpwYAAAA30lEQVR4nO3UMQoCMRBAURXLqbOFjRfxEJ5gzpYTeArvsY2FqdOvrQgLaVbB/1+ZDGHgQ/bLsuz0/w6/XkDfcVy7KKVcI2IaeaT3fo6I2dnx2Y33eLbWbu9nq6EjYsrMx8jDtdZLZt6dHZ/deI/T55lfN4ShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxgawtAQhoYwNIShIQwNYWgIQ0MYGsLQEIaGMDSEoSEMDWFoCENDGBrC0BCGhjA0hKEhDA1haAhDQxga4gWhg1ey46OS8wAAAABJRU5ErkJggg=="">",0(0.0%)
2,first_name[object],1. MICHAEL2. DAVID3. DANIEL4. CHRIS5. JOHN6. RYAN7. JASON8. MATTHEW9. ANDREW10. JAMES11. other,"2,226 (1.6%)1,995 (1.5%)1,575 (1.1%)1,486 (1.1%)1,385 (1.0%)1,365 (1.0%)1,278 (0.9%)1,246 (0.9%)1,224 (0.9%)1,184 (0.9%)122,500 (89.1%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAAsTAAALEwEAmpwYAAACNUlEQVR4nO3asW0CQRRF0V2LcNIRgauigq2NClyVA4gnxy2AJQtf6ZyYYKQr/WAf++Px2Gj6ePcD+D3xwsQLEy9MvDDxwsQL2+eclzHGea11u9/vX+9+EM/7GGOcj+P4HmOc3/0YXuNshokXJl6YeGHihYkXJl6YeGHihYkX5ttm2O4PSF3OZpizGWYSCnM2w8QLEy9MvDDxwsQLEy9MvDDxwsQL820zzCQU5myG7XPOy7Ztm5PZczIFdTmbYeKFiRcmXph4YeKFiRcmXph4YeKFndZat3c/gt8xCYU5m2EmoTCTUJizGSZemHhh4oWJFyZemHhh4oWJFyZemEkozCQU5myGmYTCTEJhzmaYeGHihYkXJl6YeGHihYkXJl6YeGEmoTCTUJizGWYSCjMJhTmbYeKFiRcmXph4YeKFiRcmXph4YeKFmYTCTEJhzmaYSSjMJBTmbIaJFyZemHhh4oWJFyZemHhh4oWJF2YSCjMJhTmbYSahMJNQmLMZJl6YeGHihYkXJl6YeGHihYkXJl6YeGH2vDB7XpizGWbPC7PnhTmbYeKFiRcmXph4YeKFiRcmXph4YeKFmYTCTEJhzmaYSSjMJBTmbIaJFyZemHhh4oWJFyZemHhh4oWJF2YSCjMJhTmbYfuc82JZ+B/WWrdXprnTGON8HMf3Xz6K51yv189Xfu9shokXJl6YeGHihYkXJl6YeGHihYkX5tvmP/Lqt02TUJizGSZemHhh4oWJFyZe2A/C4p5MZ4F89gAAAABJRU5ErkJggg=="">",1(0.0%)
3,lastname[object],1. SMITH2. LEE3. JOHNSON4. JONES5. KIM6. BROWN7. WILLIAMS8. GARCIA9. MILLER10. WILSON11. other,"713 (0.5%)394 (0.3%)392 (0.3%)388 (0.3%)384 (0.3%)368 (0.3%)330 (0.2%)318 (0.2%)312 (0.2%)271 (0.2%)133,594 (97.2%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAAsTAAALEwEAmpwYAAACPklEQVR4nO3aIW7jUBhG0eco0NQKKG/X0xVkbVlB9zO8IMbmKRppYMfk6UrnoMAnXekH/rK8Xq9B02X2AzhPvDDxwsQLEy9MvDDxwpZt2z7HGGPf96/Zj+H/XNd1vc1+BOc4m2HihYkXJl6YeGHihYkXJl6YeGHihV2P43jOfgTnLP6A1OVshl2WZfmY/QjOuYwx3mc/gnOczTDxwsQLEy9MvDDxwsQLEy9MvDDxwi5jjD+zH8E5JqEwZzPMJBRmEgpzNsPECxMvTLww8cLECxMvTLww8cLECzMJhZmEwpzNMJNQmEkozNkMEy9MvDDxwsQLEy9MvDDxwsQLEy/MJBRmEgpzNsNMQmEmoTBnM0y8MPHCxAsTL0y8MPHCxAsTL0y8MJNQmEkozNkMMwmFmYTCnM0w8cLECxMvTLww8cLECxMvTLww8cJMQmEmoTBnM8wkFGYSCnM2w8QLEy9MvDDxwsQLEy9MvDDxwsQLMwmFmYTCnM0wk1CYSSjM2QwTL0y8MPHCxAsTL0y8MPHCxAsTL0y8MHtemD0vzNkMs+eF2fPCnM0w8cLECxMvTLww8cLECxMvTLww8cJMQmEmoTBnM8wkFGYSCnM2w8QLEy9MvDDxwsQLEy9MvDDxwsQLMwmFmYTCnM2wZdu2z3Vdb7Mfwu8cx/Hc9/1rjDGu67re7vf79+xH8TuPx+Pt729nM0y8MPHCxAsTL0y8MPHCxAsTL0y8MN82Y/79tmkSCnM2w8QLEy9MvDDxwsQL+wHV9EvmjSb7vwAAAABJRU5ErkJggg=="">",8(0.0%)
4,Placing[object],1. 1350182. 1320823. 1217674. 1273225. 1256796. 1340997. 1369168. 1281509. 12355310. 13108111. other,"852 (0.6%)595 (0.4%)591 (0.4%)450 (0.3%)401 (0.3%)380 (0.3%)378 (0.3%)378 (0.3%)363 (0.3%)353 (0.3%)132,723 (96.6%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAG8AAACzCAYAAACHOUI0AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjYuMywgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy/P9b71AAAACXBIWXMAAAsTAAALEwEAmpwYAAACPklEQVR4nO3aIW7jUBhG0eco0NQKKG/X0xVkbVlB9zO8IMbmKRppYMfk6UrnoMAnXekH/rK8Xq9B02X2AzhPvDDxwsQLEy9MvDDxwpZt2z7HGGPf96/Zj+H/XNd1vc1+BOc4m2HihYkXJl6YeGHihYkXJl6YeGHihV2P43jOfgTnLP6A1OVshl2WZfmY/QjOuYwx3mc/gnOczTDxwsQLEy9MvDDxwsQLEy9MvDDxwi5jjD+zH8E5JqEwZzPMJBRmEgpzNsPECxMvTLww8cLECxMvTLww8cLECzMJhZmEwpzNMJNQmEko

In [601]:
# Verificando o "tamanho" do df
open_masc.shape

(137464, 19)

In [602]:
open_masc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137464 entries, 0 to 137463
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         137464 non-null  int64  
 1   first_name         137463 non-null  object 
 2   lastname           137456 non-null  object 
 3   Placing            137464 non-null  object 
 4   country            137464 non-null  object 
 5   continent          137464 non-null  object 
 6   Age                137464 non-null  object 
 7   affiliate          112681 non-null  object 
 8   Height and Weight  87327 non-null   object 
 9   points             137463 non-null  float64
 10  21.1               137464 non-null  object 
 11  21.1 time          8238 non-null    object 
 12  21.2               137462 non-null  object 
 13  21.2 time          65789 non-null   object 
 14  21.3               137461 non-null  object 
 15  21.3 time          15202 non-null   object 
 16  21

In [603]:
# Primeira descrição dos dados numéricos, 
open_masc.describe()

,Unnamed: 0,points
count,137464.000000,137463.000000
mean,68731.500000,266044.715029
std,39682.583039,132747.169208
min,0.000000,101.000000
25%,34365.750000,154847.000000
50%,68731.500000,274493.000000
75%,103097.250000,386686.500000
max,137463.000000,467118.000000


In [604]:
# Verificando quais são as colunas originais do df
open_masc.columns

Index(['Unnamed: 0', 'first_name', 'lastname', 'Placing', 'country',
       'continent', 'Age', 'affiliate', 'Height and Weight', 'points', '21.1',
       '21.1 time', '21.2', '21.2 time', '21.3', '21.3 time', '21.4', 'weight',
       'Unnamed: 18'],
      dtype='object')

## Tratando colunas

### Renomeando as colunas

In [605]:
open_masc =  open_masc.rename(columns={'weight': 'weight_lifted', 'Age': 'age', '21.2 time': '21.2 reps', '21.1 time': '21.1 reps', 'Placing':'placing'})
open_masc.columns

Index(['Unnamed: 0', 'first_name', 'lastname', 'placing', 'country',
       'continent', 'age', 'affiliate', 'Height and Weight', 'points', '21.1',
       '21.1 reps', '21.2', '21.2 reps', '21.3', '21.3 time', '21.4',
       'weight_lifted', 'Unnamed: 18'],
      dtype='object')

### Separando as colunas de altura e peso do atleta
Ambas as colunas são dados numéricos.

Unidade de de medidas: peso(Kg), altura(cm)

#### Altura do atleta

In [606]:
open_masc['height'] = open_masc['Height and Weight']

In [607]:
def define_height(value):
    x = None
    if type(value) == str:
        a = value.split()
        if a[1] == 'in':
            x = int(a[0]) * 2.54
        elif a[1] == 'cm':
            x = int(a[0])
    return x

In [608]:
open_masc['height'] = open_masc['height'].apply(define_height)

#### Peso do atleta

In [609]:
open_masc['weight'] = open_masc['Height and Weight']

In [610]:
def define_weight(value):
    x = None
    if type(value) == str:
        a = value.split()
        if a[-1] == 'lb':
            x = int(a[-2]) / 2.2
        elif a[-1] == 'kg':
            x = int(a[-2])
    return x

In [611]:
open_masc['weight'] = open_masc['weight'].apply(define_weight)

Removendo a coluna de peso e altura 

In [612]:
open_masc.drop(['Height and Weight'], axis=1, inplace=True)

### Transformando a coluna de idade em numérico

Durante o processo para realizar a transformação da coluna idade (de string para numérico) observou-se inconsistência nos dados da linha 57645, optando-se pela remoção da mesma

In [613]:
open_masc.iloc[57645]

Unnamed: 0                         57645
first_name                         NICOL
lastname                             NaN
placing                             MARC
country                            57646
continent                 United Kingdom
age                               Europe
affiliate                         Age 41
points                               NaN
21.1                              237470
21.1 reps             48384th (236 reps)
21.2                                 NaN
21.2 reps             63386th (214 reps)
21.3                                 NaN
21.3 time             68919th (126 reps)
21.4                                 NaN
weight_lifted          56781st (176 lbs)
Unnamed: 18      Weight lifted: 176 lbs.
height                               NaN
weight                               NaN
Name: 57645, dtype: object

In [614]:
open_masc.drop(labels=57645, axis=0, inplace=True, errors='raise')

In [615]:
def transforma_idade(texto):
    if texto == "":
        x = 0
    else:
        texto.split(" ")
        x = texto[1]
    return(texto[-2:])

In [616]:
open_masc['age'] = open_masc['age'].apply(transforma_idade)


In [617]:
# Convertendo a coluna age (idade) de 'string' para numérico
open_masc['age'] = pd.to_numeric(open_masc['age'])


##### Transformando em numérico as colunas com números de reps 

In [618]:
# Incluindo texto nos campos nulos de acordo com a coluna
values = {'21.1 reps': '0', '21.2 reps': '0', '21.3 time': '0'}
open_masc.fillna(value = values, inplace=True) 

In [619]:
def remove_reps(t):
    t = str(t)
    if ('th' in t):
        t = t.split()
        t = t[1]
        t = t.replace('(', "")
    elif ('st' in t):
        t = t.split()
        t = t[1]
        t = t.replace('(', "")
    else:
        t = t.replace(' reps', "")
        t = t.replace(" db snatch", "")
        t = t.replace(" snatch", "")
        t = t.replace(" front squat", "")
        t = t.replace(" squat", "")
    return t

In [620]:
open_masc['21.1 reps'] = open_masc['21.1 reps'].apply(remove_reps)
open_masc['21.2 reps'] = open_masc['21.2 reps'].apply(remove_reps)
open_masc['21.3 time'] = open_masc['21.3 time'].apply(remove_reps)
open_masc['21.1 reps'] = pd.to_numeric(open_masc['21.1 reps'])
open_masc['21.2 reps'] = pd.to_numeric(open_masc['21.2 reps'])
open_masc['21.3 time'] = pd.to_numeric(open_masc['21.3 time'])

### Transformando a coluna de peso levantado
Considerando unidade de medida Kg

In [621]:
# Preenchendo pesos nullos com valor zero, por que considera-se que o atleta não realizou a prova
open_masc['weight_lifted'] = open_masc['weight_lifted'].fillna('0')

In [622]:
# Iniciando o processo para converter weight_lift para numerico
# Função para remover a string 'lbs' e converter peso para quilos
def substitui_lbs(value):
    x = value.replace(' lbs', '')
    x = x.replace('Weight lifted:', '')
    x = float(x) / 2.205
    return(round(x,2))

In [623]:
# Função para remover a string 'kg'
def substitui_kg(value):
    x = value.replace(' kg', '')
    return(x)

In [624]:
open_masc['weight_lifted'] = open_masc['weight_lifted'].apply(substitui_kg)


In [625]:
#Funaçao para substituir valores inválidos por peso zero
def remove_peso_invalido(value):
    if ('(' in value):
        value = '0'
    return value

In [626]:
open_masc['weight_lifted'] = open_masc['weight_lifted'].apply(remove_peso_invalido)

In [627]:
open_masc['weight_lifted'] = open_masc['weight_lifted'].apply(substitui_lbs)


In [628]:
# Convertendo a coluna weight lifted de 'string' para numérico
open_masc['weight_lifted'] = pd.to_numeric(open_masc['weight_lifted'])

In [629]:
open_masc['weight_lifted']

0         143.76
1         143.31
2         156.46
3         143.31
4         141.50
           ...  
137459      0.00
137460      0.00
137461      0.00
137462      0.00
137463      0.00
Name: weight_lifted, Length: 137463, dtype: float64

### Transformando em dado numérico (tempo) as colunas '21.1', '21.2' e '21.3'

In [630]:
open_masc[150:160]

,Unnamed: 0,first_name,lastname,placing,country,continent,age,affiliate,points,21.1,21.1 reps,21.2,21.2 reps,21.3,21.3 time,21.4,weight_lifted,Unnamed: 18,height,weight
150,150,COLBY,THOMASON,151,United States,North America,30,Koda CrossFit,2413.0,615th (14:32),605,242nd (10:31),225,1314th (10:41),180,242nd (301 lbs),136.51,NaN,175.26,86.363636
151,151,SIGURDUR,JONSSON,152,Iceland,Europe,30,CrossFit XY,2416.0,413th (14:10),605,1511th (11:39),225,367th (9:30),180,125th (309 lbs),140.14,NaN,175.00,90.000000
152,152,SIMON,MANTYLA,153,Sweden,Europe,30,CrossFit Mejeriet,2453.0,1534th (548 reps),0,122nd (10:12),225,671st (9:56),180,126th (309 lbs),140.14,NaN,181.00,93.000000
153,153,DAN,BAILEY,154,United States,North America,37,NaN,2470.0,535th (14:25),605,261st (10:33),225,591st (9:50),180,1083rd (277 lbs),125.62,NaN,170.18,81.818182
154,154,JASON,CARROLL,155,United States,North America,32,CrossFit Tustin,2474.0,51st (12:42),605,149th (10:17),225,40th (8:22),180,2234th (268 lbs),121.54,NaN,177.80,81.818182
155,155,KEVIN,STEINHAUS,156,United States,North America,33,CrossFit CLT,2483.0,445th (14:16),605,518th (10:54),225,800th (10:06),180,720th (285 lbs),129.25,NaN,187.96,97.727273
156,156,SEAN,SWEENEY,157,United States,North America,29,CrossFit Powerstroke,2486.0,196th (13:38),605,677th (11:03),225,235th (9:11),180,1378th (275 lbs),124.72,NaN,175.26,87.272727
157,157,PAWEL,KOZAK,158,Poland,Europe,31,CrossFit Mokotow,2497.0,47th (12:39),605,190th (10:24),225,284th (9:19),180,1976th (271 lbs),122.90,NaN,183.00,96.000000
158,158,NIKLAS,HECHT,159,United States,North America,30,NaN,2540.0,362nd (14:04),605,615th (10:59),225,483rd (9:42),180,1080th (277 lbs),125.62,NaN,177.80,93.181818
159,159,MICHAL,WESOLOWSKI,160,Poland,Europe,28,NaN,2570.0,1610th (543 reps),0,212th (10:27),225,615th (9:52),180,133rd (308 lbs),139.68,NaN,NaN,NaN


In [631]:
a = open_masc['21.1']

In [632]:
from datetime import time
# help(time)

In [651]:
def arruma_tempo(t):
    if (('r' or '-') in t):
        t = '0'
    t = t.replace('(', '').replace(')', '')
    t = str(t).split()
    # x = t[-1].split(':')
    # delta = time(minute=int(x[0]), second=int(x[1]))
    return t[-1]

In [652]:
open_masc['21.1'] = open_masc['21.1'].apply(arruma_tempo)
open_masc['21.1'] = open_masc['21.1'].apply(arruma_tempo)
open_masc['21.1'] = open_masc['21.1'].apply(arruma_tempo)

0         11:55
1             0
2         13:02
3         11:25
4         12:45
          ...  
137459       --
137460       --
137461       --
137462       --
137463       --
Name: 21.1, Length: 137463, dtype: object

# PAREI AQUI

In [635]:
open_masc.tail()

,Unnamed: 0,first_name,lastname,placing,country,continent,age,affiliate,points,21.1,21.1 reps,21.2,21.2 reps,21.3,21.3 time,21.4,weight_lifted,Unnamed: 18,height,weight
137459,137459,JACK,NORTON,137351,United Kingdom,Europe,31,NaN,467118.0,117347th (--),0,111374th (--),0,101151st (--),0,137246th (--),0.0,NaN,NaN,NaN
137460,137460,JAMES,DOUGHTY,137351,United States,North America,32,NaN,467118.0,117347th (--),0,111374th (--),0,101151st (--),0,137246th (--),0.0,NaN,NaN,NaN
137461,137461,CHAD,ROTT,137351,United States,North America,46,Polarize CrossFit,467118.0,117347th (--),0,111374th (--),0,101151st (--),0,137246th (--),0.0,NaN,NaN,NaN
137462,137462,NATE,RICHARDS,137351,United States,North America,39,CrossFit Home Office Scotts Valley,467118.0,117347th (--),0,111374th (--),0,101151st (--),0,137246th (--),0.0,NaN,193.04,93.181818
137463,137463,ANDREW,WEINSTEIN,137351,United States,North America,51,NaN,467118.0,117347th (--),0,111374th (--),0,101151st (--),0,137246th (--),0.0,NaN,NaN,NaN


In [636]:
open_masc.describe()

,Unnamed: 0,age,points,21.1 reps,21.2 reps,21.3 time,weight_lifted,height,weight
count,137463.000000,137463.000000,137463.000000,137463.000000,137463.000000,137463.000000,137463.000000,82114.000000,86501.000000
mean,68731.580651,34.745233,266044.715029,36.252555,107.557677,19.835148,61.066333,179.671160,85.127761
std,39682.716113,8.325646,132747.169208,143.592124,112.390879,56.363010,40.773070,48.668526,37.040680
min,0.000000,16.000000,101.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.454545
25%,34365.500000,29.000000,154847.000000,0.000000,0.000000,0.000000,0.000000,174.000000,77.272727
50%,68732.000000,34.000000,274493.000000,0.000000,0.000000,0.000000,74.830000,178.000000,84.090909
75%,103097.500000,41.000000,386686.500000,0.000000,225.000000,0.000000,91.610000,182.880000,90.909091
max,137463.000000,54.000000,467118.000000,605.000000,225.000000,180.000000,1666.210000,1953.260000,10000.000000


### Limpeza os dados

##### limpeza


In [637]:
# Conferindo os continentes listados, 
# verificou-se a necessidade de remover dados inconsistentes
continent = np.array(open_masc['continent'])
unique_continent = np.unique(continent)
print(unique_continent)

['Africa' 'Asia' 'Europe' 'North America' 'Oceania' 'South America']


In [638]:
# Visualizando todos os países listados
# Verificou-se a necessidade de remover uma linha com dados inconsistentes
country = np.array(open_masc['country'])
unique_country = np.unique(country)
print("Total de países participantes: ", len(unique_country))
print(unique_country)

Total de países participantes:  156
['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola' 'Argentina'
 'Armenia' 'Australia' 'Austria' 'Azerbaijan' 'Bahamas' 'Bahrain'
 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize' 'Benin' 'Bolivia'
 'Bosnia and Herzegovina' 'Botswana' 'Brazil' 'Brunei Darussalam'
 'Bulgaria' 'Cambodia' 'Cameroon' 'Canada' 'Cape Verde' 'Chile' 'China'
 'Colombia' 'Congo, The Democratic Republic of the' 'Costa Rica' 'Croatia'
 'Cyprus' 'Czech Republic' "Côte d'Ivoire" 'Denmark' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Estonia' 'Ethiopia'
 'Fiji' 'Finland' 'France' 'Georgia' 'Germany' 'Ghana' 'Greece'
 'Guatemala' 'Guinea' 'Guyana' 'Haiti' 'Honduras' 'Hungary' 'Iceland'
 'India' 'Indonesia' 'Iran' 'Iraq' 'Ireland' 'Israel' 'Italy' 'Jamaica'
 'Japan' 'Jordan' 'Kazakhstan' 'Kenya' 'Korea, Republic of' 'Kosovo'
 'Kuwait' 'Kyrgyzstan' 'Laos' 'Latvia' 'Lebanon' 'Libya' 'Liechtenstein'
 'Lithuania' 'Luxembourg' 'Macedonia' 'Madagascar

In [639]:
open_masc.isnull().sum()

Unnamed: 0            0
first_name            1
lastname              7
placing               0
country               0
continent             0
age                   0
affiliate         24783
points                0
21.1                  0
21.1 reps             0
21.2                  1
21.2 reps             0
21.3                  2
21.3 time             0
21.4                  2
weight_lifted         0
Unnamed: 18      137461
height            55349
weight            50962
dtype: int64

#### Seleção de colunas

In [640]:
# Novo Dataframe apenas com as colunas que serão analisadas
df_masc = open_masc[['id', 'first_name', 'lastname', 'height', 'weight', 'country', 'continent',
       'age', 'affiliate', 'points', '21.1',
       '21.2', '21.3', '21.4', 'weight_lifted']]
df_masc[100500:100510]

KeyError: "['id'] not in index"

#### Países e continentes

In [ ]:
# removendo inconsistencias de paises e continentes
filtro1 = df_masc['country'] != '57646'
df_masc = df_masc[filtro1]

#### Idades

,id,first_name,lastname,placing,country,continent,age,affiliate,height,weight,points,21.1,21.2,21.3,21.4,weight_lifted
0,0,JEFFRE,ADLER,1,Canada,North America,27,CrossFit Wonderland,69 in,197 lb,101,20th (11:55),8th (9:14),27th (8:15),46th (317 lbs),317
1,1,SCOTT,PANCHIK,2,United States,North America,33,CrossFit Mentality,69 in,187 lb,141,33rd (12:25),47th (9:52),5th (7:48),56th (316 lbs),316
2,2,TRAVIS,MEAD,3,United States,North America,34,Iron Valley CrossFit,73 in,205 lb,165,87th (13:02),24th (9:38),48th (8:26),6th (345 lbs),345
3,3,SAXON,PANCHIK,4,United States,North America,25,CrossFit Cliffside,69 in,180 lb,217,5th (11:25),68th (9:59),87th (8:41),57th (316 lbs),316
4,4,RICHARD,FRONING JR.,5,United States,North America,33,CrossFit Mayhem,69 in,194 lb,254,58th (12:45),91st (10:04),5th (7:48),100th (312 lbs),312
5,5,NOAH,OHLSEN,6,United States,North America,30,Peak 360 CrossFit,67 in,190 lb,272,11th (11:41),21st (9:33),2nd (7:44),238th (301 lbs),301
6,6,SAMUEL,COURNOYER,7,Canada,North America,25,CrossFit Mayhem,180 cm,205 lb,276,158th (13:27),79th (10:02),20th (8:09),19th (328 lbs),328
7,7,COLE,GREASHABER,8,United States,North America,21,CrossFit Lee's Summit (LS),72 in,206 lb,403,82nd (13:01),68th (9:59),46th (8:25),207th (303 lbs),303
8,8,AGUSTIN,RICHELME,9,Argentina,South America,24,NaN,173 cm,83 kg,417,30th (12:19),137th (10:15),1st (7:40),249th (300 lbs),300
9,9,OLEG,LASCENKO,10,United Kingdom,Europe,23,CrossFit Abu Dhabi,175 cm,92 kg,428,53rd (12:43),22nd (9:34),102nd (8:46),251st (300 lbs),300


#### peso levantado

#### pontos

In [ ]:
df_masc.head(5)

,id,first_name,lastname,placing,country,continent,age,affiliate,height,weight,points,21.1,21.2,21.3,21.4,weight_lifted
0,0,JEFFRE,ADLER,1,Canada,North America,27,CrossFit Wonderland,69 in,197 lb,101,20th (11:55),8th (9:14),27th (8:15),46th (317 lbs),143.764172
1,1,SCOTT,PANCHIK,2,United States,North America,33,CrossFit Mentality,69 in,187 lb,141,33rd (12:25),47th (9:52),5th (7:48),56th (316 lbs),143.310658
2,2,TRAVIS,MEAD,3,United States,North America,34,Iron Valley CrossFit,73 in,205 lb,165,87th (13:02),24th (9:38),48th (8:26),6th (345 lbs),156.462585
3,3,SAXON,PANCHIK,4,United States,North America,25,CrossFit Cliffside,69 in,180 lb,217,5th (11:25),68th (9:59),87th (8:41),57th (316 lbs),143.310658
4,4,RICHARD,FRONING JR.,5,United States,North America,33,CrossFit Mayhem,69 in,194 lb,254,58th (12:45),91st (10:04),5th (7:48),100th (312 lbs),141.496599


In [ ]:
df_masc['points']

0                   101
1                   141
2                   165
3                   217
4                   254
              ...      
137459    117347th (--)
137460    117347th (--)
137461    117347th (--)
137462          467.118
137463    117347th (--)
Name: points, Length: 137463, dtype: object

In [ ]:
# Inicio do processo para converter points para numérico
def remove_pontos_invalidos(value):
    if ('lb' in value):
        x = "0"
    elif ('kg' in value):
        x = '0'
    elif ('t' in value):
        x = '0'
    elif ('d' in value):
        x = '0'
    else:
        x = value
    return x

In [ ]:
df_masc.head(2)

,id,first_name,lastname,placing,country,continent,age,affiliate,height,weight,points,21.1,21.2,21.3,21.4,weight_lifted
0,0,JEFFRE,ADLER,1,Canada,North America,27,CrossFit Wonderland,69 in,197 lb,101,20th (11:55),8th (9:14),27th (8:15),46th (317 lbs),143.764172
1,1,SCOTT,PANCHIK,2,United States,North America,33,CrossFit Mentality,69 in,187 lb,141,33rd (12:25),47th (9:52),5th (7:48),56th (316 lbs),143.310658


In [ ]:
df_masc['points'] = df_masc['points'].apply(remove_pontos_invalidos)
# df_masc['points']

In [ ]:
# Convertendo a coluna points lifted de 'string' para numérico
df_masc['points'] = pd.to_numeric(df_masc['points'])
df_masc['points']

0         101.000
1         141.000
2         165.000
3         217.000
4         254.000
           ...   
137459      0.000
137460      0.000
137461      0.000
137462    467.118
137463      0.000
Name: points, Length: 137463, dtype: float64

In [ ]:
df_masc['points'] = df_masc['points'].astype(float)
df_masc['points']

0         101.000
1         141.000
2         165.000
3         217.000
4         254.000
           ...   
137459      0.000
137460      0.000
137461      0.000
137462    467.118
137463      0.000
Name: points, Length: 137463, dtype: float64

In [ ]:
df_masc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 137463 entries, 0 to 137463
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             137463 non-null  int64  
 1   first_name     137462 non-null  object 
 2   lastname       137456 non-null  object 
 3   placing        137463 non-null  object 
 4   country        137463 non-null  object 
 5   continent      137463 non-null  object 
 6   age            137463 non-null  int64  
 7   affiliate      112680 non-null  object 
 8   height         87194 non-null   object 
 9    weight        119239 non-null  object 
 10  points         137463 non-null  float64
 11  21.1           103061 non-null  object 
 12  21.2           106707 non-null  object 
 13  21.3           101173 non-null  object 
 14  21.4           117636 non-null  object 
 15  weight_lifted  137463 non-null  float64
dtypes: float64(2), int64(2), object(12)
memory usage: 17.8+ MB


#### Removendo linhas nulas

In [ ]:
df_masc.isna().sum()

id                   0
first_name           1
lastname             7
placing              0
country              0
continent            0
age                  0
affiliate        24783
height           50269
 weight          18224
points               0
21.1             34402
21.2             30756
21.3             36290
21.4             19827
weight_lifted        0
dtype: int64

In [ ]:
# Encontrando quem tem first_name nulo
df_masc[pd.isna(df_masc['first_name'])]

,id,first_name,lastname,placing,country,continent,age,affiliate,height,weight,points,21.1,21.2,21.3,21.4,weight_lifted
93138,93138,NaN,YEONGDON,93.139,"Korea, Republic of",Asia,28,CrossFit 660,NaN,359096,0.0,NaN,225,NaN,NaN,0.0


In [ ]:
# Encontrando quem tem lastname nulo
df_masc[pd.isna(df_masc['lastname'])]

,id,first_name,lastname,placing,country,continent,age,affiliate,height,weight,points,21.1,21.2,21.3,21.4,weight_lifted
2040,2040,HYEONJONG,NaN,2.041,"Korea, Republic of",Asia,31,CrossFit Zest,172 lb,NaN,17.676,3890th (441),3593rd (12:29),5047th (12:56),5146th (252 lbs),114.285714
7732,7732,YOUNGSEON,NaN,7.733,"Korea, Republic of",Asia,23,Golden Crown CrossFit Seodaemun,NaN,NaN,48.734,5280th (395),10317th (13:59),10886th (14:54),22251st (215 lbs),97.505669
19017,19017,TAKSEOK,NaN,19.018,"Korea, Republic of",Asia,41,CrossFit Geumchon,170 cm,75 kg,96.625,11332nd (387),40439th (18:10),22176th (155),22678th (215 lbs),97.505669
73598,73598,MICHAEL,NaN,73.599,Canada,North America,51,CrossFit Calgary,NaN,290309,0.000,NaN,NaN,NaN,175,0.000000
81080,81080,JONGMYUNG,NaN,81.081,"Korea, Republic of",Asia,28,CrossFit Lagom,NaN,315173,0.000,NaN,NaN,180,155,0.000000
86324,86324,DAVID,NaN,86.325,United States,North America,37,CrossFit East Oahu,70 in,195 lb,333.850,87458th (115),92161st (145),78825th (98),75406th (147 lbs),66.666667
133060,133060,BRIAN,NaN,132.987,United States,North America,33,NaN,69 in,165 lb,459.276,117347th (--),111374th (--),101151st (--),129404th (--),0.000000


In [ ]:
# Incluindo texto nos campos nulos de acordo com a coluna
values = {'lastname': '-', 'first_name': '-', 'affiliate': 'not_affiliate'}
df_masc.fillna(value = values, inplace=True) 

In [ ]:
df_masc.isna().sum()
# Nenhum valor null

id                   0
first_name           0
lastname             0
placing              0
country              0
continent            0
age                  0
affiliate            0
height           50269
 weight          18224
points               0
21.1             34402
21.2             30756
21.3             36290
21.4             19827
weight_lifted        0
dtype: int64